## Train model

#### Import data

In [190]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from alibi.explainers import AnchorTabular
from sklearn.model_selection import train_test_split

dataset = load_iris()
feature_names = dataset.feature_names
class_names = list(dataset.target_names)

#### Train our model

In [191]:
idx = 145
X_train,Y_train = dataset.data[:idx,:], dataset.target[:idx]
X_test, Y_test = dataset.data[idx+1:,:], dataset.target[idx+1:]

X = dataset.data
Y = dataset.target

np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=50)

In [192]:
idx = 0
print('Input: ', X_test[idx])
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

Input:  [6.3 2.5 5.  1.9]
Prediction:  virginica


#### Export model for deployment

In [193]:
import joblib
with open("model.joblib", "wb") as file:
    joblib.dump(clf, file)

In [194]:
!ls | grep model

model.joblib


## Deploy model

In [195]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: multiclass-model
spec:
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
      logger:
        url: http://default-broker.seldon-logs.svc.cluster.local:80/
        mode: all
    name: default
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/multiclass-model configured


In [196]:
!kubectl get pods | grep multiclass-model

multiclass-model-default-0-classifier-776746677-m5xr6   2/2     Running   0          12h
seldon-multiclass-model-metrics-57b6749597-d9dk2        1/1     Running   0          42h


### Send requests to visualise system performance

In [197]:
import time

while True:
    for x in X:
        pred_req = {"data":{"ndarray":[x.tolist()]}}

        pred_resp = requests.post(f"{url}/predictions", json=pred_req)

        time.sleep(0.1)

KeyboardInterrupt: 

### To send feedback first send requests and capture id

In [198]:
import time

puids = []

for x in X:
    pred_req = {"data":{"ndarray":[x.tolist()]}}
    pred_resp = requests.post(f"{url}/predictions", json=pred_req)
    
    puid_seldon = pred_resp.headers.get("seldon-puid")
    puids.append(puid_seldon)
    time.sleep(0.1)

#### Send feedback to visualise statistical performance (actual)

In [161]:
y_one_hot = np.eye(Y.max() + 1)[Y] # Covert to one hot
for puid, y in zip(puids, y_one_hot):
    data = {
        "truth": {
            'data': {
                'names': ['t:0', 't:1', 't:2'], 
                'ndarray': [y.tolist()]
            }
        }
    }
    requests.post(f"{url}/feedback", json=data, headers={"seldon-puid": puid})
    time.sleep(0.5)

#### Send feedback to visualise statistical performance (Skewed)

In [200]:
y_one_hot = np.eye(Y.max() + 1)[Y] # Covert to one hot
y_rand = np.array(y_one_hot)
np.random.shuffle(y_rand)
while True:
    for puid, y in zip(puids, y_rand):
        data = {
            "truth": {
                'data': {
                    'names': ['t:0', 't:1', 't:2'], 
                    'ndarray': [y.tolist()]
                }
            }
        }
        requests.post(f"{url}/feedback", json=data, headers={"seldon-puid": puid})
        time.sleep(0.1)

KeyboardInterrupt: 

### Explainer
Now we can train and deploy an explainer.

#### Train explainer

In [201]:
predict_fn = lambda x: clf.predict_proba(x)

explainer = AnchorTabular(predict_fn, feature_names)

explainer.fit(X_train, disc_perc=(25, 50, 75))

AnchorTabular(meta={
    'name': 'AnchorTabular',
    'type': ['blackbox'],
    'explanations': ['local'],
    'params': {'seed': None, 'disc_perc': (25, 50, 75)}
})

#### Choose datapoint and prediction to explain

In [202]:
idx = 0
print('Input: ', X_test[idx])
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

Input:  [6.3 2.5 5.  1.9]
Prediction:  virginica


#### Run explanation on datapoint

In [203]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: petal width (cm) > 1.80 AND sepal width (cm) <= 2.80
Precision: 0.96
Coverage: 0.32


In [204]:
import dill
with open("explainer.dill", "wb") as file:
    dill.dump(explainer, file)

In [205]:
!ls | grep ex

explainer.dill


#### Deploy Explainer

In [206]:
%%bash

kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: iris-explainer
spec:
  annotations:
    seldon.io/rest-timeout: "100000"
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    explainer:
      type: AnchorTabular
      modelUri: gs://seldon-models/sklearn/iris-0.23.2/anchor
    name: default
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/iris-explainer configured


#### Choose prediction to explain (the same one)

In [207]:
print('Input: ', X_test[idx])

Input:  [6.3 2.5 5.  1.9]


#### Send explanation request

In [208]:
explain_url = "http://localhost:80/seldon/seldon/iris-explainer-explainer/default/api/v1.0"

In [209]:
print(X_test[idx].tolist())

[6.3, 2.5, 5.0, 1.9]


In [210]:
pred_req_1 = {"data":{"ndarray":  [X_test[idx].tolist()] }}
explanation = requests.post(f"{explain_url}/explain", json=pred_req_1)
print(explanation)

<Response [200]>


In [211]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: petal width (cm) > 1.80 AND sepal width (cm) <= 2.80
Precision: 0.98
Coverage: 0.32


In [98]:
!curl -X POST -H 'Content-Type: application/json' \
    -d '{"data": {"names": ["text"], "ndarray": [[6.3, 2.5, 5.0, 1.9]]}}' \
    http://localhost:80/seldon/seldon/iris-explainer-explainer/default/api/v1.0/explain 

{"meta": {"name": "AnchorTabular", "type": ["blackbox"], "explanations": ["local"], "params": {"seed": null, "disc_perc": [25, 50, 75], "kwargs": {}, "verbose_every": 1, "verbose": false, "cache_margin": 1000, "binary_cache_size": 10000, "n_covered_ex": 10, "min_samples_start": 100, "max_anchor_size": null, "stop_on_first": false, "beam_size": 1, "coverage_samples": 10000, "batch_size": 100, "tau": 0.15, "delta": 0.1, "threshold": 0.95}}, "data": {"anchor": ["petal width (cm) > 1.80", "petal length (cm) > 4.20"], "precision": 0.9873873873873874, "coverage": 0.496551724137931, "raw": {"feature": [3, 2], "mean": [0.6406388642413487, 0.9873873873873874], "precision": [0.6406388642413487, 0.9873873873873874], "coverage": [0.20689655172413793, 0.496551724137931], "examples": [{"covered_true": [[5.1, 2.5, 3.0, 2.5], [6.0, 2.9, 4.5, 1.9], [6.0, 2.2, 4.0, 2.1], [5.7, 3.0, 4.2, 2.2], [5.9, 3.0, 4.2, 2.1], [5.4, 3.0, 4.5, 2.5], [7.6, 3.0, 6.6, 1.9], [6.8, 2.8, 4.8, 2.0], [6.0, 3.4, 4.5, 2.3], [7